In [1]:
import numpy as np
import os
import pandas as pd
import pickle
import xgboost as xgb

from run_xgb.check_importance import check_importance
from run_xgb.reads import read_config
from run_xgb.run import run_xgb
from scipy import stats
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, mean_absolute_error

pd.options.display.max_columns=500
pd.options.display.max_rows=500
pd.options.display.max_colwidth=500
np.set_printoptions(precision=3)

In [2]:
df = pd.read_pickle('../data/input/owid-covid-data.pickle')
config = read_config()

In [3]:
isnull_vacc_or_death = (df['people_fully_vaccinated_per_hundred'].isnull()) \
                        | (df['new_deaths_per_million'].isnull())

df = df[~isnull_vacc_or_death]
print(df.shape)

(37603, 67)


In [4]:
def get_tomorrow_label(df, config):
    
    df_lst = []
    for country in df['iso_code'].unique():
        country_df = df[df['iso_code']==country]
        country_df['new_deaths_per_million_tomorrow'] = country_df['new_deaths_per_million'].shift(-1).values
        country_df = country_df.iloc[:-1]

        df_lst.append(country_df)
    
    res_df = pd.concat(df_lst)[['date'] + config['train_cols'] + [config['label_col']]]
    res_df = pd.concat(df_lst)
    res_df.reset_index(drop=True, inplace=True)
    return res_df

In [5]:
shift_df = get_tomorrow_label(df, config)
shift_df['date'] = pd.to_datetime(shift_df['date'])

/home/saiko/ML/git_pvgorshenin/runners/run_xgb/venv/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [6]:
print(shift_df.shape)

(37389, 68)


### Model check

In [7]:
train, val = train_test_split(shift_df, train_size=.7)

In [8]:
class Splitter():
    
    def __init__(self):
        self.n_splits = 1
    
    def split(self, df):
        q7 = df['date'].quantile(.7)
        df.reset_index(inplace=True)
        yield (df[df['date'] <= q7].index.values, df[df['date'] > q7].index.values)


In [9]:
for train_index, test_index in Splitter().split(shift_df):
    pass

In [10]:
res = run_xgb(train=train,
              val=val,
              train_cols=config['train_cols'],
              train_labels=train[config['label_col']],
              val_labels=val[config['label_col']],
              booster_params=config['booster_params'],
              train_params=config['train_params'],
              log_params=config['log_params'],
              kfold=Splitter(),
              metric=mean_absolute_error)

../data/result/02-16-14
[0]	train-rmse:5.22697	val-rmse:4.82257
[10]	train-rmse:4.48660	val-rmse:4.30901
[20]	train-rmse:4.00454	val-rmse:3.98791
[30]	train-rmse:3.69732	val-rmse:3.78079
[40]	train-rmse:3.50018	val-rmse:3.63641
[50]	train-rmse:3.37454	val-rmse:3.54085
[60]	train-rmse:3.28647	val-rmse:3.48902
[70]	train-rmse:3.22316	val-rmse:3.46595
[80]	train-rmse:3.16956	val-rmse:3.44226
[90]	train-rmse:3.13026	val-rmse:3.42835
[100]	train-rmse:3.10637	val-rmse:3.41896
[110]	train-rmse:3.08462	val-rmse:3.41388
[120]	train-rmse:3.06233	val-rmse:3.41896
[121]	train-rmse:3.05981	val-rmse:3.42028


### Feature importance checker

In [16]:
with open('../data/result/02-16-12/models/model0.pickle', 'rb') as f:
    model = pickle.load(f)

In [17]:
result = check_importance(shift_df,
                          model,
                          input_col='people_fully_vaccinated_per_hundred',
                          input_change_method='rel',
                          target_measure_method='mass_rel',
                          agg_method='mean',
                          increase_koeff=25)

Start preds mean --> 2.52    changed preds mean --> 2.40
-0.0487


In [18]:
shift_df[config['label_col']].mean()

2.6841397202385724

In [19]:
d = model.get_score(importance_type='gain')
pd.DataFrame({'feature': d.keys(), 'score': d.values()}).sort_values(by='score', ascending=False)[:20]

,feature,score
2,new_deaths_per_million,2586.627441
0,aged_70_older,544.917725
1,people_fully_vaccinated_per_hundred,223.294739


In [3]:
544 / (2586 + 544 + 223)

0.1622427676707426